In [ ]:
import pandas as pd
import numpy as np
import sys
sys.path.append('./src') # Add src directory to Python path

from data_preprocessing import preprocess_data, split_data
from modeling import train_evaluate_regression_model, train_evaluate_classification_model
from model_interpretation import plot_shap_summary, get_feature_importance

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.linear_model import LogisticRegression # For classification task
from sklearn.ensemble import RandomForestClassifier # For classification task

- --- Load Data ---

In [ ]:
df = pd.read_csv('data/raw/historical_insurance_claims.csv')
df['TransactionDate'] = pd.to_datetime(df['TransactionDate'])

- --- 1. Claim Severity Prediction (Regression) ---

In [ ]:
print("--- Claim Severity Prediction Model ---")
# Filter for policies with claims
df_claims_only = df[df['TotalClaims'] > 0].copy()

# Define target and features for Claim Severity
target_severity = 'TotalClaims'
# List features to exclude. Be careful not to include features that leak information about the target.
# 'CalculatedPremiumPerTerm' could be a target or a feature. For claim severity, it might be a strong feature.
# For optimal premium, it would be a target.
features_to_exclude_severity = ['UnderwrittenCoverID', 'PolicyID', 'TransactionDate', 'PostalCode',
                               'TotalPremium', 'CalculatedPremiumPerTerm', # Can be strong features
                               'ClaimOccurred', 'ClaimFrequency', 'Margin', 'LossRatio'] # Exclude derived claim metrics

X_severity_raw, y_severity, preprocessor_severity, severity_features = \
    preprocess_data(df_claims_only, target_severity, features_to_exclude=features_to_exclude_severity)

# Transform features
X_severity_transformed = preprocessor_severity.fit_transform(X_severity_raw)
# Get feature names after one-hot encoding if needed for SHAP
feature_names_severity = preprocessor_severity.get_feature_names_out()


# Split data
X_train_sev, X_test_sev, y_train_sev, y_test_sev = split_data(
    pd.DataFrame(X_severity_transformed, columns=feature_names_severity), y_severity
)

# Train and Evaluate Models
models_severity = {
    'Linear Regression': LinearRegression(),
    'Random Forest Regressor': RandomForestRegressor(random_state=42),
    'XGBoost Regressor': XGBRegressor(random_state=42)
}

results_severity = {}
for name, model in models_severity.items():
    trained_model, metrics = train_evaluate_regression_model(model, X_train_sev, y_train_sev, X_test_sev, y_test_sev, name)
    results_severity[name] = {'model': trained_model, 'metrics': metrics}

# Compare Models
print("\n--- Claim Severity Model Comparison ---")
for name, res in results_severity.items():
    print(f"{name}: RMSE={res['metrics']['rmse']:.4f}, R2={res['metrics']['r2']:.4f}")

# Feature Importance and Interpretability for the best-performing model (e.g., XGBoost)
best_severity_model = results_severity['XGBoost Regressor']['model']
plot_shap_summary(best_severity_model, X_test_sev, feature_names_severity, plot_type='bar')
plot_shap_summary(best_severity_model, X_test_sev, feature_names_severity, plot_type='dot')